# Chapter 11: Algebraic datatypes

## Multiple choice

In [1]:
data Weekday =
    Monday
    | Tuesday
    | Wednesday
    | Thursday
    | Friday

`Weekday` is a type with five data constructors

In [2]:
f Friday = "Miller Time"

:t f
-- f :: Weekday -> String

f :: Weekday -> [Char]

Types defined with the `data` keyword must begin with a capital letter.

The function `g xs = xs !! (length xs - 1)` delivers the final element of `xs`.

## Ciphers

---

In [3]:
import Data.Char

In [4]:
punctuations = " "
base = ord 'a'
end = ord 'z'
size = end - base + 1

In [5]:
import Data.Maybe

data LetterOrPunctuation = Letter Char | Punctuation Char deriving Show

unLetter :: LetterOrPunctuation -> Char
unLetter (Letter x) = x
unLetter (Punctuation x) = x

getLetter :: Char -> Maybe LetterOrPunctuation
getLetter x
    | x `elem` punctuations = (Just . Punctuation) x
    | l < base = Nothing
    | l > end = Nothing
    | otherwise = (Just . Letter) x
    where l = ord x

getLetterFromInt :: Int -> Maybe LetterOrPunctuation
getLetterFromInt = getLetter . chr

unsafeGetLetter :: Char -> LetterOrPunctuation
unsafeGetLetter = fromJust . getLetter

unsafeGetLetterFromInt :: Int -> LetterOrPunctuation
unsafeGetLetterFromInt = fromJust . getLetterFromInt

caesarLetter :: Int -> LetterOrPunctuation -> LetterOrPunctuation
caesarLetter _ (Punctuation x) = Punctuation x
caesarLetter shift (Letter x) = (unsafeGetLetterFromInt . f . ord) x where
    f i = (i - base + shift) `mod` size + base
    
caesarLetter 3 (Letter 'a')

Letter 'd'

In [6]:
getLetters :: String -> Maybe [LetterOrPunctuation]
getLetters = traverse getLetter

unsafeGetLetters :: String -> [LetterOrPunctuation]
unsafeGetLetters = fromJust . getLetters

unLetters :: [LetterOrPunctuation] -> String
unLetters = fmap unLetter

caesar :: Int -> [LetterOrPunctuation] -> [LetterOrPunctuation]
caesar shift = fmap (caesarLetter shift)

unLetters <$> caesar 3 <$> getLetters "qwer"
unLetters <$> caesar 3 <$> getLetters "qw er"

Just "tzhu"

Just "tz hu"

In [7]:
uncaesar :: Int -> [LetterOrPunctuation] -> [LetterOrPunctuation]
uncaesar shift = caesar (-shift)

unLetters . uncaesar 3 <$> caesar 3 <$> getLetters "qwer"
unLetters . uncaesar 3 <$> caesar 3 <$> getLetters "qw er"

Just "qwer"

Just "qw er"

---

In [8]:
zipIgnoringPunctuation :: [a] -> [LetterOrPunctuation] -> [(Maybe a, LetterOrPunctuation)]
zipIgnoringPunctuation = f where
    f [] _ = []
    f _ [] = []
    f xs (y@(Punctuation _):yt) = (Nothing, y):f xs yt
    f (x:xt) (y:yt) = (Just x, y):f xt yt

In [9]:
zipIgnoringPunctuation [1, 2, 3, 4] <$> getLetters "qwer"
zipIgnoringPunctuation [1, 2, 3, 4] <$> getLetters "qw er"

Just [(Just 1,Letter 'q'),(Just 2,Letter 'w'),(Just 3,Letter 'e'),(Just 4,Letter 'r')]

Just [(Just 1,Letter 'q'),(Just 2,Letter 'w'),(Nothing,Punctuation ' '),(Just 3,Letter 'e'),(Just 4,Letter 'r')]

In [10]:
toShift :: LetterOrPunctuation -> Maybe Int
toShift (Letter x) = Just $ ord x - base
toShift (Punctuation _) = Nothing

In [18]:
import Control.Monad (join)
import Data.Bifunctor (first)

vigenere' :: (Int -> Int) -> [LetterOrPunctuation] -> [LetterOrPunctuation] -> [LetterOrPunctuation]
vigenere' mapShift word s = encoded where
    shifts = cycle . fmap (fmap mapShift . toShift) $ word
    dirtyPairs = zipIgnoringPunctuation shifts s
    pairs = fmap (first join) dirtyPairs
--     f Nothing l = l
--     f (Just s) l = caesarLetter s l
    f s l = fromMaybe l (caesarLetter <$> s <*> pure l)
    encoded = fmap (uncurry f) pairs
    
vigenere :: [LetterOrPunctuation] -> [LetterOrPunctuation] -> [LetterOrPunctuation]
vigenere = vigenere' id

In [19]:
unLetters <$> (vigenere <$> getLetters "ally" <*> getLetters "meet at dawn")

Just "mppr ae oywy"

In [33]:
unvigenere :: [LetterOrPunctuation] -> [LetterOrPunctuation] -> [LetterOrPunctuation]
unvigenere = vigenere' (*(-1))

input = getLetters "meet at dawn"
keyword = getLetters "ally"

code = (vigenere <$> keyword <*>)
uncode = (unvigenere <$> keyword <*>)

coded = code input
uncoded = uncode coded

unLetters <$> uncoded

Just "meet at dawn"

## As-patterns